# Graph update example

In [ ]:
%pylab inline

In [ ]:
from tinc import *
tclient = TincClient()

In [ ]:
tclient.synchronize()

In [ ]:
imageBuffer = tclient.get_disk_buffer("graph") 

In [ ]:
imageBuffer._path

In [ ]:
import random
import matplotlib.pyplot as plt
import threading

def update_graph(parameter_value):
    #print("Parameter value " + str(parameter_value))
    data = [random.random() * parameter_value for i in range(10)]
    with threading.Lock():
        fname = imageBuffer.get_filename_for_writing()
        f = plt.figure()
        plt.title("Random numbers with range 0->" + str(parameter_value))
        plt.plot(data)
        #print("Update " + fname)
        plt.savefig(fname)
        plt.close()
        f.clf()
        del f
        imageBuffer.done_writing_file(fname)

In [ ]:
update_graph(1)

In [ ]:
param = tclient.get_parameter("internalValuesDim")

In [ ]:
param

In [ ]:
param.register_callback(update_graph)

In [ ]:
param.value = 0.1

In [ ]:
pserver.monitor_server()

In [ ]:
pserver.stop()

In [ ]:
param.value

In [ ]:
pwd